# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

%load_ext autoreload
%autoreload 2

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#cf): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours): Number of crew or vessel hours spent at sea
 - [Number of Tows](#tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-costs): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#npv): Project NPV calculator
 - [PySAM-Powered Results](#pysam): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 96.0%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.98,0.98,0.82,0.93,0.98,0.98,0.98,0.98,0.96,0.97,0.98,0.98,0.93,0.90,0.96,0.89,0.98,0.98,0.89,0.98,0.97,0.98,0.98,0.98,0.98,0.98,0.98,0.90,0.98,0.94,0.99,0.95,0.96,0.98,0.90,0.93,0.98,0.98,0.91,0.93,0.97,0.98,0.95,0.94,0.97,0.98,0.94,0.98,0.96,0.98,0.98,0.98,0.98,0.98,0.98,0.97,0.93,0.97,0.98,0.98,0.96,0.95,0.98,0.98,0.95,0.97,0.96,0.87,0.88,0.98,0.94,0.98,0.99,0.98,0.97,0.98,0.98,0.98,0.96,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.96
2004,0.96
2005,0.96
2006,0.96
2007,0.96
2008,0.96
2009,0.96
2010,0.97
2011,0.96


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.97
2,0.96
3,0.96
4,0.95
5,0.95
6,0.96
7,0.96
8,0.97
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.97
     3          0.96
     4          0.93
     5          0.94
     6          0.99
     7          1.00
     8          0.99
     9          0.97
     10         0.95
     11         0.93
     12         0.95
2004 1          0.96
     2          0.97
     3          0.96
     4          0.95
     5          0.94
     6          0.95
     7          0.95
     8          0.99
     9          0.97
     10         0.97
     11         0.96
     12         0.96

<a id="cf"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.47,0.47,0.39,0.45,0.46,0.47,0.47,0.47,0.46,0.46,0.47,0.47,0.44,0.43,0.45,0.43,0.47,0.47,0.41,0.47,0.46,0.47,0.47,0.47,0.47,0.47,0.47,0.43,0.47,0.45,0.47,0.45,0.45,0.47,0.43,0.45,0.47,0.47,0.44,0.45,0.46,0.47,0.45,0.45,0.46,0.47,0.45,0.47,0.46,0.47,0.47,0.47,0.46,0.47,0.47,0.46,0.43,0.46,0.47,0.47,0.46,0.45,0.47,0.47,0.46,0.46,0.45,0.41,0.42,0.47,0.45,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.45,0.47


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.40
2004,0.45
2005,0.47
2006,0.45
2007,0.49
2008,0.50
2009,0.45
2010,0.41
2011,0.48


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.61
2,0.50
3,0.50
4,0.41
5,0.38
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.36
     3          0.36
     4          0.50
     5          0.33
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.42
     11         0.52
     12         0.54
2004 1          0.54
     2          0.51
     3          0.50
     4          0.39
     5          0.29
     6          0.37
     7          0.26
     8          0.42
     9          0.57
     10         0.55
     11         0.48
     12         0.48

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.97
2005,1.03
2006,1.00
2007,1.01
2008,1.00
2009,1.00
2010,1.01
2011,0.98


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.97
2,1.00
3,1.03
4,1.02
5,1.04
6,0.99
7,1.00
8,1.02
9,1.00


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.85
     2                 1.10
     3                 0.95
     4                 1.01
     5                 1.06
     6                 0.99
     7                 1.00
     8                 1.00
     9                 0.96
     10                1.03
     11                0.82
     12                1.10
2004 1                 0.55
     2                 1.03
     3                 1.35
     4                 1.21
     5                 1.03
     6                 1.11
     7                 1.05
     8                 1.01
     9                 0.97
     10                0.90
     11                0.78
     12                0.87

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $422,677.02/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,173,501.90","81,089,201.39"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"12,923,015.12"
2004,"7,201,548.61"
2005,"7,151,131.29"
2006,"7,070,897.85"
2007,"14,734,156.75"
2008,"14,031,694.18"
2009,"12,540,937.50"
2010,"12,264,510.16"
2011,"6,968,678.82"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,083.33","543,010.42","543,010.42",0.00,"370,312.50"
2,"496,052.08","496,125.00","495,250.00","150,391.71","2,184,206.60"
3,"541,989.58","541,989.58","542,864.58","123,500.00","4,543,750.00"
4,"524,635.42","524,635.42","524,635.42","71,947.66","936,836.81"
5,"542,500.00","542,500.00","542,500.00","14,250.00","14,112,135.42"
6,"525,000.00","525,000.00","525,000.00",0.00,"7,840,906.25"
7,"542,500.00","542,500.00","542,500.00","105,474.74","8,204,378.47"
8,"542,500.00","542,500.00","542,500.00","508,437.80","10,723,437.50"
9,"525,000.00","525,000.00","525,000.00","199,500.00","4,245,097.22"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       4,628,836.81  
     6                   52,500.00                  0.00         676,562.50  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00            101,591.50               0.00  
     9                   52,500.00            171,000.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00         777,243.06  
     12                  54,250.00                  0.00       4,650,000.00  
2004 1                   54,250.00                  0.00         220,312.50  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00       3,259,736.11  
     8                   54,250.00                  0.00       1,800,000.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

<a id="utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.88,0.95


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.94,0.97
2004,1.00,1.00,1.00,0.00,0.94
2005,1.00,1.00,1.00,0.85,0.94
2006,1.00,1.00,1.00,0.83,0.91
2007,1.00,1.00,1.00,0.94,0.98
2008,1.00,1.00,1.00,0.85,0.96
2009,1.00,1.00,1.00,0.00,0.97
2010,1.00,1.00,1.00,0.83,0.92
2011,1.00,1.00,1.00,0.00,0.97


<a id="vessel-crew-hours"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"119,643.23"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 1,Crew Transfer Vessel 3,Crew Transfer Vessel 2,Heavy Lift Vessel,Field Support Vessel
0,2003,"10,723.22","2,068.50","1,994.75","2,031.75","4,014.72",613.50
1,2004,"12,607.00","3,719.50","3,575.25","3,604.25","1,708.00",0.00
2,2005,"13,573.13","3,536.50","3,586.00","3,543.50","1,803.13","1,104.00"
3,2006,"11,598.75","2,977.25","2,886.75","2,940.75","1,690.00","1,104.00"
4,2007,"13,684.44","2,971.25","3,158.25","2,964.50","3,989.44",601.00
5,2008,"13,490.97","3,014.25","2,937.50","2,771.00","3,664.22","1,104.00"
6,2009,"11,731.95","2,619.00","2,642.25","2,702.50","3,768.20",0.00
7,2010,"13,029.77","2,912.50","2,742.50","2,756.50","3,514.27","1,104.00"
8,2011,"9,372.39","2,547.25","2,423.00","2,496.75","1,905.39",0.00
9,2012,"9,831.62","2,453.00","2,450.75","2,446.25","1,840.68",640.94


<a id="tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

<a id="equipment-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,316,106.77"
Repair,"21,114,197.92"
Crew Transfer,"1,128,078.13"
Site Travel,0.00
Mobilization,"7,500,000.00"
Weather Delay,"30,367,519.31"
No Requests,"17,996,453.63"
Not in Shift,"21,020,128.79"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,316,106.77","2,316,106.77"
Repair,0.00,0.00,0.00,"21,114,197.92","21,114,197.92"
Crew Transfer,0.00,0.00,0.00,"1,128,078.13","1,128,078.13"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"7,500,000.00","7,500,000.00"
Weather Delay,0.00,0.00,0.00,"30,367,519.31","30,367,519.31"
No Requests,0.00,0.00,0.00,"17,996,453.63","17,996,453.63"
Not in Shift,0.00,0.00,0.00,"21,020,128.79","21,020,128.79"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      71,895.83
     Repair        3,207,831.60
     Crew Transfer   148,046.88
     Site Travel           0.00
     Mobilization  1,000,000.00
     Weather Delay 4,218,651.91
     No Requests   1,355,265.63
     Not in Shift  2,921,323.28
2004 Maintenance     384,398.44
     Repair        1,554,026.04

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                   
1     Maintenance    150,682.29
      Repair         231,549.48
      Crew Transfer   40,640.63
      Site Travel          0.00
      Mobilization         0.00
      Weather Delay  569,966.15
      No Requests    653,096.35
      Not in Shift   353,481.77
2     Maintenance    173,505.21
      Repair         435,487.85

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,052.08
           Crew Transfer    2,369.79
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   16,242.19
           No Requests    117,250.00
           Not in Shift     8,367.19
     2     Maintenance          0.00
           Repair          25,283.85
           Crew Transfer    2,697.92
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   11,885.42
           No Requests     95,575.52
           Not in Shift    11,557.29
     3     Maintenance          0.00
           Repair          22,403.65
           Crew Transfer    2,697.92
           Site Travel          0.00

<a id="component-costs"></a>
## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"112,706,711.73"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   51,260,828.91   
          maintenance            0.00              0.00    2,316,106.77   
          repair                 0.00              0.00   21,114,197.92   

                         total_cost  
component action                     
turbine   delay       51,260,828.91  
          maintenance  2,316,106.77  
          repair      21,114,197.92

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"4,233,000.00",0.00,"10,563,593.24","14,796,593.24"
2004,turbine,"3,789,500.00",0.00,"4,494,273.44","8,283,773.44"
2005,turbine,"4,013,000.00",0.00,"5,249,938.58","9,262,938.58"
2006,turbine,"2,926,500.00",0.00,"3,596,080.73","6,522,580.73"
2007,turbine,"4,318,500.00",0.00,"12,804,822.05","17,123,322.05"
2008,turbine,"4,146,500.00",0.00,"10,380,180.56","14,526,680.56"
2009,turbine,"3,788,500.00",0.00,"10,018,307.29","13,806,807.29"
2010,turbine,"4,531,000.00",0.00,"8,293,250.87","12,824,250.87"
2011,turbine,"2,999,500.00",0.00,"5,537,022.57","8,536,522.57"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"1,585,000.00",0.00,"1,336,039.06","2,921,039.06"
2,turbine,"1,363,000.00",0.00,"2,736,106.12","4,099,106.12"
3,turbine,"1,877,000.00",0.00,"5,461,515.63","7,338,515.63"
4,turbine,"1,912,500.00",0.00,"1,468,326.39","3,380,826.39"
5,turbine,"5,659,000.00",0.00,"12,217,147.57","17,876,147.57"
6,turbine,"4,617,000.00",0.00,"7,866,770.83","12,483,770.83"
7,turbine,"3,919,500.00",0.00,"5,413,687.50","9,333,187.50"
8,turbine,"4,980,000.00",0.00,"7,931,792.23","12,911,792.23"
9,turbine,"1,825,500.00",0.00,"4,742,761.29","6,568,261.29"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         51,000.00              0.00       45,500.00   
     2     turbine         60,000.00              0.00       49,674.48   
     3     turbine         55,000.00              0.00       40,085.94   
     4     turbine         53,000.00              0.00       41,908.85   
     5     turbine      1,734,500.00              0.00    4,175,940.97   
     6     turbine        372,000.00              0.00      708,208.33   
     7     turbine         54,000.00              0.00       27,635.42   
     8     turbine        289,000.00              0.00      146,463.90   
     9     turbine        116,500.00              0.00       77,929.69   
     10    turbine         91,000.00              0.00       65,770.83   
     11    turbine        143,500.00              0.00      392,086.81   
     12    turbine      1,213,500.00              0.00    4,792,388.02   
2004 1     turbine        550,000.00              0.00      382,187.50   
     2     turbine        149,500.00              0.00      152,250.00   
     3     turbine        224,500.00              0.00      162,750.00   
     4     turbine        292,500.00              0.00      154,875.00   
     5     turbine        273,000.00              0.00      161,000.00   
     6     turbine        147,500.00              0.00       85,914.06   
     7     turbine      1,380,500.00              0.00    2,748,088.54   
     8     turbine        103,500.00              0.00       74,393.23   
     9     turbine        129,000.00              0.00      103,085.94   
     10    turbine        188,500.00              0.00      149,479.17   
     11    turbine        177,500.00              0.00      157,500.00   
     12    turbine        173,500.00              0.00      162,750.00   
2005 1     turbine        211,000.00              0.00      159,250.00   
     2     turbine        164,000.00              0.00      144,375.00   
     3     turbine        169,000.00              0.00      159,250.00   
     4     turbine        222,500.00              0.00      154,875.00   

                       total_cost  
year month component               
2003 1     turbine      96,500.00  
     2     turbine     109,674.48  
     3     turbine      95,085.94  
     4     turbine      94,908.85  
     5     turbine   5,910,440.97  
     6     turbine   1,080,208.33  
     7     turbine      81,635.42  
     8     turbine     435,463.90  
     9     turbine     194,429.69  
     10    turbine     156,770.83  
     11    turbine     535,586.81  
     12    turbine   6,005,888.02  
2004 1     turbine     932,187.50  
     2     turbine     301,750.00  
     3     turbine     387,250.00  
     4     turbine     447,375.00  
     5     turbine     434,000.00  
     6     turbine     233,414.06  
     7     turbine   4,128,588.54  
     8     turbine     177,893.23  
     9     turbine     232,085.94  
     10    turbine     337,979.17  
     11    turbine     335,000.00  
     12    turbine     336,250.00  
2005 1     turbine     370,250.00  
     2     turbine     308,375.00  
     3     turbine     328,250.00  
     4     turbine     377,375.00

<a id="fixed-costs"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [47]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [48]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low").head(28)

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
     6      131,507.51
     7      135,891.09
     8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09
2004 1      135,891.09
     2      127,123.92
     3      135,891.09
     4      131,507.51
     5      135,891.09
     6      131,507.51
     7      135,891.09
     8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09
2005 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [49]:
metrics.opex("project")

,OpEx
0,"154,343,215.30"


In [50]:
metrics.opex("annual")

,OpEx
year,
2003,"18,756,023.12"
2004,"12,595,440.20"
2005,"12,764,139.29"
2006,"11,597,405.85"
2007,"20,652,664.75"
2008,"19,782,585.76"
2009,"17,929,445.50"
2010,"18,395,518.16"
2011,"11,568,186.82"


In [51]:
metrics.opex("monthly")

,OpEx
month,
1,"4,943,327.57"
2,"6,425,579.54"
3,"9,530,004.65"
4,"5,810,265.78"
5,"22,771,796.32"
6,"15,347,981.32"
7,"15,215,764.12"
8,"19,198,286.21"
9,"9,160,172.29"


In [52]:
metrics.opex("month-year").head(28)

OpEx
year month             
2003 1       351,172.34
     2       329,740.34
     3       353,641.09
     4       342,007.51
     5     6,661,977.90
     6     1,337,570.01
     7       352,641.09
     8       689,232.60
     9       576,507.51
     10      389,641.09
     11    1,209,750.56
     12    6,162,141.09
2004 1     1,068,953.59
     2       428,873.92
     3       523,141.09
     4       581,507.51
     5       571,641.09
     6       436,507.51
     7     4,938,877.20
     8     2,202,141.09
     9       418,007.51
     10      487,141.09
     11      466,507.51
     12      472,141.09
2005 1       509,641.09
     2       433,740.34
     3       467,641.09
     4       511,507.51

In [53]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2004 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2005 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2006 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2007 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2008 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2009 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2010 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2011 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2012 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2013 1              0

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [54]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"233,196.86","76,234.25","69,075.00",532.00
major repair,"52,564.00","1,457.75","1,026.47",22.00
major replacement,"170,838.49","10,215.65","170,838.49",62.00
manual reset,"203,139.00","37,682.50","13,439.50","5,734.00"
medium repair,"13,141.66","10,489.75","8,437.50",212.00
minor repair,"67,084.29","31,859.00","15,334.75","2,256.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [55]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,633,464,936.00"


In [56]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,633,464,936.00","123,700,896.00","123,065,407.00","101,718,473.00","117,276,117.50","122,132,131.00","122,534,025.50","122,774,127.00","123,493,251.50","120,279,221.00","121,232,191.00","123,798,405.50","122,628,342.50","116,579,281.50","113,576,175.50","118,355,273.50","112,906,684.00","123,289,547.00","123,785,621.50","108,985,441.50","122,398,928.00","121,451,966.00","123,258,298.50","123,567,292.00","122,756,020.50","122,521,340.00","123,634,130.00","122,969,547.50","113,376,080.00","122,426,528.50","118,575,607.50","124,125,379.00","118,865,943.50","119,388,277.00","122,983,558.00","114,287,460.00","117,104,121.50","122,840,359.50","122,703,768.50","114,800,748.00","117,851,559.00","122,053,234.50","122,691,573.50","118,709,202.50","117,694,325.00","121,947,329.00","122,439,870.50","119,214,716.00","122,856,493.50","121,363,201.50","123,198,020.00","122,902,134.00","123,060,657.50","122,212,553.00","122,878,606.00","123,111,437.00","122,254,199.00","114,157,548.50","121,183,570.00","122,538,168.50","123,483,935.50","120,346,023.50","119,191,076.50","122,598,000.00","122,943,522.50","119,857,460.50","121,218,341.00","119,581,813.50","108,806,470.50","109,481,493.50","123,565,465.50","117,313,964.00","123,172,403.50","123,984,311.00","122,945,539.00","122,466,147.00","122,751,657.00","122,995,853.00","123,591,707.00","119,455,527.00","123,249,860.00"


In [57]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"849,112,842.00"
2004,"942,458,719.00"
2005,"985,058,654.00"
2006,"950,394,092.00"
2007,"1,027,743,848.00"
2008,"1,053,487,415.00"
2009,"951,720,619.50"
2010,"865,236,838.00"
2011,"1,018,650,369.50"


In [58]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,083,130,224.00"
2,"822,209,145.00"
3,"900,091,492.50"
4,"703,060,290.00"
5,"684,456,104.50"
6,"555,584,895.50"
7,"584,845,949.50"
8,"644,486,203.50"
9,"793,344,618.50"


In [59]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     111,558,186.50
     2      57,786,805.00
     3      65,025,301.00
     4      87,116,689.50
     5      58,824,031.00
     6      53,870,924.50
     7      53,907,866.00
     8      53,430,497.50
     9      46,882,079.50
     10     74,591,497.00
     11     90,312,971.00
     12     95,805,993.50
2004 1      96,452,750.50
     2      85,566,971.00
     3      90,066,390.50
     4      66,711,630.00
     5      50,949,625.50
     6      64,345,437.00
     7      47,192,509.50
     8      74,150,033.50
     9      99,298,126.50
     10     98,651,805.00
     11     82,587,465.00
     12     86,485,975.00

<a id="npv"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [60]:
metrics.npv("project")

,NPV
0,"193,789,105.74"


In [61]:
metrics.opex("annual")

,OpEx
year,
2003,"18,756,023.12"
2004,"12,595,440.20"
2005,"12,764,139.29"
2006,"11,597,405.85"
2007,"20,652,664.75"
2008,"19,782,585.76"
2009,"17,929,445.50"
2010,"18,395,518.16"
2011,"11,568,186.82"


In [62]:
metrics.opex("monthly")

,OpEx
month,
1,"4,943,327.57"
2,"6,425,579.54"
3,"9,530,004.65"
4,"5,810,265.78"
5,"22,771,796.32"
6,"15,347,981.32"
7,"15,215,764.12"
8,"19,198,286.21"
9,"9,160,172.29"


In [63]:
metrics.opex("month-year").head(28)

OpEx
year month             
2003 1       351,172.34
     2       329,740.34
     3       353,641.09
     4       342,007.51
     5     6,661,977.90
     6     1,337,570.01
     7       352,641.09
     8       689,232.60
     9       576,507.51
     10      389,641.09
     11    1,209,750.56
     12    6,162,141.09
2004 1     1,068,953.59
     2       428,873.92
     3       523,141.09
     4       581,507.51
     5       571,641.09
     6       436,507.51
     7     4,938,877.20
     8     2,202,141.09
     9       418,007.51
     10      487,141.09
     11      466,507.51
     12      472,141.09
2005 1       509,641.09
     2       433,740.34
     3       467,641.09
     4       511,507.51

<a id="pysam"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [64]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [65]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [66]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [67]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [68]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
